<a href="https://colab.research.google.com/github/dks0101/machine_learing/blob/main/code/last_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')
import random
import numpy as np
import pandas as pd
import chardet
torch.manual_seed(0)

seq_length = 24
data_dim = 1
hidden_dim = 10
output_dim = 24
learning_rate = 0.06
iterations = 350
#train data 입력
train_df = pd.read_csv('/content/drive/MyDrive/기후 train/data_tr_city.csv', encoding='euc-kr')
train_df.columns = ['datetime', 'D']
#test data 입력
with open('/content/drive/MyDrive/기후 train/data_ts_city.csv', 'rb') as f:
    result = chardet.detect(f.read())
test_df = pd.read_csv('/content/drive/MyDrive/기후 train/data_ts_city.csv', encoding=result['encoding'], error_bad_lines=False)
test_df.columns = ['datetime', 'D']
#데이터 전처리
#'D'에서 1000보다 큰 값들을 찾고 값을 na으로 만들어주기
train_df.loc[train_df['D'] > 1000, 'D'] = np.NaN
#이상치 선형 보간 - 총 3개의 이상치
index = train_df[train_df.D == max(train_df.D)].index
train_df.D[index] = np.NaN
index = train_df[train_df.D == min(train_df.D)].index
train_df.D[index] = np.NaN
# 앞서 min(data_tr.적산차)인 행을 nan값으로 변경했으므로 그 다음 min값이 리턴된다.
index = train_df[train_df.D== min(train_df.D)].index
train_df.D[index] = np.NaN
# 이상치를 NaN으로 대체한 이후에 linear 보간을 수행 & 결측치도 linear보간 수행
train_df['D'] = train_df['D'].interpolate(method='linear')
from sklearn.preprocessing import MinMaxScaler
# Min-Max Scaling을 위한 객체 생성
scaler = MinMaxScaler()
# 'D' 열 선택 및 2차원 배열로 변환
D = train_df['D'].values.reshape(-1, 1)
# Min-Max Scaling 적용
scaled_D = scaler.fit_transform(D)
# 정규화된 값을 다시 'D' 열에 할당
train_df['D'] = scaled_D
# 'D' 열 선택 및 2차원 배열로 변환
test_D = test_df['D'].values.reshape(-1, 1)
# 학습 데이터의 스케일러 객체를 사용하여 테스트 데이터에도 변환 적용
scaled_test_D = scaler.transform(test_D)
# 정규화된 값을 다시 'D' 열에 할당
test_df['D'] = scaled_test_D
# datetime을 index로 지정
train_df = train_df.set_index('datetime')
test_df = test_df.set_index('datetime')
# 시계열 데이터 생성 함수
def build_dataset(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range(0, len(time_series) - seq_length):
        _x = time_series.values[i:i+seq_length, :]
        _y = time_series.values[i+seq_length, [-1]]
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

trainX, trainY = build_dataset(train_df, seq_length)
testX, testY = build_dataset(test_df, seq_length)

trainX_tensor = torch.FloatTensor(trainX)
trainY_tensor = torch.FloatTensor(trainY)

testX_tensor = torch.FloatTensor(testX)
testY_tensor = torch.FloatTensor(testY)
#model
class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim, layers):
    super (Net, self).__init__()
    self.rnn = torch.nn. LSTM(input_dim, hidden_dim, num_layers=layers, batch_first=True)
    self.fc= torch.nn. Linear(hidden_dim, output_dim, bias=True)

  def forward(self, x):
    x, _status = self.rnn(x)
    x = self.fc(x[:, -1])
    return x

net = Net(data_dim, hidden_dim, output_dim, 1)

# loss & optimizer setting
criterion = torch.nn.L1Loss()
optimizer = optim.Adam (net.parameters(), lr=learning_rate)

for i in range(iterations):
    optimizer.zero_grad()
    outputs = net(trainX_tensor)
    loss = criterion(outputs, trainY_tensor)
    loss.backward()
    optimizer.step()

    # MAE Loss 값 출력
    scaled_loss = loss.item()
    original_loss = scaler.inverse_transform([[scaled_loss]])[0][0]

    # MAE 계산
    mae = torch.mean(torch.abs(outputs - trainY_tensor))
    scaled_mae = mae.item()
    original_mae = scaler.inverse_transform([[scaled_mae]])[0][0]

    test_outputs = net(testX_tensor)
    test_loss = criterion(test_outputs, testY_tensor)
    scaled_test_loss = test_loss.item()
    original_test_loss = scaler.inverse_transform([[scaled_test_loss]])[0][0]

    test_mae = torch.mean(torch.abs(test_outputs - testY_tensor))
    scaled_test_mae = test_mae.item()
    original_test_mae = scaler.inverse_transform([[scaled_test_mae]])[0][0]
    print("Iteration:", i+1, " Train Loss:", original_loss, "Train MAE:", original_mae, "Test Loss:", original_test_loss, "Test MAE:", original_test_mae)

In [ ]:
#그래프로 결과 출력
plt.figure(figsize=(200, 10))
original_testY = scaler.inverse_transform(testY_tensor.numpy())
plt.plot(original_testY)
prediction = scaler.inverse_transform(net(testX_tensor).data.numpy())
plt.plot(prediction)
plt.legend(['original', 'prediction'])
plt.show()

# DataFrame 생성
df = pd.DataFrame(prediction)
# DataFrame 출력
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df.tail(5))
